# Test inference on ESP32 microcontroller

In [476]:
from __future__ import absolute_import, division, print_function, unicode_literals

import glob
import serial
import time

import numpy as np
import tensorflow as tf

from PIL import Image
import matplotlib.pyplot as plt

In [477]:
try:
  %tensorflow_version 2.x
except:
  pass

In [478]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [479]:
# x_train = x_train / 255.0
# x_test = x_test / 255.0

In [480]:
x_train[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

In [481]:
# INPUT_IMAGE_SIZE = 14
INPUT_IMAGE_SIZE = 32

# Path to TensorFlow Lite model file
tflite_model_file = '../models/resnet8-model-2.tflite'

# Path to images folder
# path = 'test_images_14x14/'
# file_format = ".jpg"

# SERIAL_PORT_NAME = '/dev/ttyUSB0'
# SERIAL_PORT_NAME = '/dev/ttyUSB1'
# SERIAL_PORT_NAME = '/dev/ttyUSB2'
# SERIAL_PORT_NAME = '/dev/ttyUSB3'
# SERIAL_PORT_NAME = '/dev/ttyUSB4'
SERIAL_PORT_NAME = '/dev/ttyUSB5'

EXPECTED_PRECISION = 0.000001

In [482]:
def init_serial(port_name):
    # Configure and open serial port
    port = serial.Serial(
        port=port_name,
        baudrate=115200,
        bytesize=serial.EIGHTBITS,
        parity=serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE,
        xonxoff=False,
        rtscts=False,
        dsrdtr=False
    )
    
    port.reset_input_buffer()
    port.reset_output_buffer()
    return port

In [483]:
def send_to_mcu(port, data):
    bytes_written = port.write(data)
    return bytes_written

In [484]:
def read_result_from_mcu(port):
    line = port.read_until()   # read a '\n' terminated line
    return line

In [485]:
def infer_with_TF_lite(interpreter, input_details, output_details, raw_image):
    # Get input size
    input_shape = input_details[0]['shape']
    # print(input_shape, input_shape[:2], input_shape[1:3])
    # size = input_shape[:2] if len(input_shape) == 3 else input_shape[1:3]

    # Preprocess image
    # print(input_shape)
    # raw_image = raw_image.resize(size)
    img = np.array(raw_image, dtype=np.float32)
    # print(img.shape)

    # Normalize image
    img = img / 255.

    # Add a batch dimension and a dimension because we use grayscale format
    # Reshape from (INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE) to (1, INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE, 1)
    input_data = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
    
    # Point the data to be used for testing
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run the interpreter
    interpreter.invoke()

    # Obtain results
    predictions = interpreter.get_tensor(output_details[0]['index'])[0]
    
    return predictions

In [486]:
def infer_with_MCU(raw_image):
    img = np.array(raw_image, dtype=np.uint8)
    bytes_sent = send_to_mcu(serial_port, img)
    
    response_str = read_result_from_mcu(serial_port)
    response_str = response_str.decode("utf-8")
    predictions = np.fromstring(response_str, dtype=np.float32, sep=',')
    return predictions
    # return None

In [487]:
def compare_results(result_tfl, result_mcu):
    for i in range(len(result_tfl)):
        if abs(result_tfl[i] - result_mcu[i]) > EXPECTED_PRECISION :
            return False
        else:
            pass
    return True

In [488]:
# Create Interpreter (Load TFLite model).
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
# Allocate tensors
interpreter.allocate_tensors()

In [489]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

### Be sure your ESP32 board connected to development computer.

In [490]:
# Get list of present serial ports
!dmesg | grep -e tty

dmesg: read kernel buffer failed: Operation not permitted


In [491]:
# Configure and open serial port
serial_port = init_serial(SERIAL_PORT_NAME)

# Check which port was really used
print("Opened serial port : {0}".format(serial_port.name))        

Opened serial port : /dev/ttyUSB5


In [492]:
# files = [files for files in glob.glob(path + "*" + file_format, recursive=False)]
# print("Found {0} {1} files".format(len(files), file_format))

### Run inference on TensorFlow Lite model and on MCU. Compare results.

In [493]:
# for f in files:
i = 0
right_preds = 0
for raw_image in x_test[:10]:
    # Read image
    # raw_image = Image.open(f).convert('L')
    
    # result_tfl = infer_with_TF_lite(interpreter, input_details, output_details, raw_image)

    result_mcu = infer_with_MCU(raw_image)

    # status = compare_results(result_tfl, result_mcu)
    # status_str = "Ok" if status else "Reults NOT eqal!"
    # print("For image: {0} - {1}".format(i, status_str))
    # print("tfl pc:", result_tfl)
    # print(result_mcu.shape, result_mcu)
    
    print(f"tfl esp32 image({i}): y_pred = {np.argmax(result_mcu)} | y_test = {y_test[i][0]} -> {np.argmax(result_mcu) == y_test[i][0]}")

    if np.argmax(result_mcu) == y_test[i][0]:
        right_preds += 1
    i += 1

print("General Accuracy: ", right_preds/i)

tfl esp32 image(0): y_pred = 3 | y_test = 3 -> True
tfl esp32 image(1): y_pred = 8 | y_test = 8 -> True
tfl esp32 image(2): y_pred = 8 | y_test = 8 -> True
tfl esp32 image(3): y_pred = 8 | y_test = 0 -> False
tfl esp32 image(4): y_pred = 6 | y_test = 6 -> True
tfl esp32 image(5): y_pred = 6 | y_test = 6 -> True
tfl esp32 image(6): y_pred = 1 | y_test = 1 -> True
tfl esp32 image(7): y_pred = 6 | y_test = 6 -> True
tfl esp32 image(8): y_pred = 3 | y_test = 3 -> True
tfl esp32 image(9): y_pred = 1 | y_test = 1 -> True
General Accuracy:  0.9


In [494]:
serial_port.close()